Project Name - VisionGen


🎯 This project generates captions for images and videos using a vision-language model (BLIP), and converts them into speech using gTTS — creating a multi-sensory AI experience.


In [ ]:
!pip install transformers accelerate torchvision opencv-python pyttsx3 sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [1]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
from google.colab import files
import cv2
import matplotlib.pyplot as plt

# Load the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda" if torch.cuda.is_available() else "cpu")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [2]:
# Image upload - Captioning
print("📷 Upload an image file")
uploaded_img = files.upload()
img_path = next(iter(uploaded_img))
image = Image.open(img_path).convert('RGB')

inputs = processor(images=image, return_tensors="pt").to(model.device)

out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)

print(f"\n🖼️ Image Caption: {caption}")


📷 Upload an image file


Saving nature.jpg to nature.jpg

🖼️ Image Caption: a waterfall surrounded by rocks and trees


In [6]:
# Video uploading
print("\n🎬 Upload a video file")
uploaded_vid = files.upload()
vid_path = next(iter(uploaded_vid))

cap = cv2.VideoCapture(vid_path)


🎬 Upload a video file


Saving SampleVideo_1280x720_1mb.mp4 to SampleVideo_1280x720_1mb.mp4


In [7]:
# Extract Frames
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()
print(f"\n📹 Extracted {len(frames)} frames from the video.")


📹 Extracted 132 frames from the video.


In [8]:
captions = []

# Caption every 5th frame
for i in range(0, len(frames), 5):
    img = Image.fromarray(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB))
    inputs = processor(images=img, return_tensors="pt").to(model.device)

    # Beam search for better results
    out = model.generate(**inputs, num_beams=5, max_length=50, early_stopping=True)
    caption = processor.decode(out[0], skip_special_tokens=True)

    captions.append((i, caption))
    print(f"🖼️ Frame {i}: {caption}")


🖼️ Frame 0: a rabbit is sitting in a hole in the grass
🖼️ Frame 5: a rabbit is sitting in a hole in the grass
🖼️ Frame 10: a small animal in the middle of a grassy field
🖼️ Frame 15: a small animal in the middle of a grassy field
🖼️ Frame 20: an animal in the middle of a grassy field
🖼️ Frame 25: a rabbit sitting in the middle of a grassy field
🖼️ Frame 30: a white rabbit in a grassy field
🖼️ Frame 35: an image of an animal in a grassy field
🖼️ Frame 40: a rabbit sitting in the grass next to a tree
🖼️ Frame 45: a rabbit sitting in the grass next to a tree
🖼️ Frame 50: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated
🖼️ Frame 55: the ratty ratty in ratty ratty ratty ratty ratty ratty ratty rat
🖼️ Frame 60: a cartoon bear is sitting in the grass
🖼️ Frame 65: ratty in ratty the ratty ratty ratty ratty ratty ratty ratty rat
🖼️ Frame 70: a cartoon bear is standing in the 

In [9]:
with open("video_frame_captions.txt", "w") as f:
    for idx, cap in captions:
        f.write(f"Frame {idx}: {cap}\n")

print("\n✅ Captions saved to video_frame_captions.txt")



✅ Captions saved to video_frame_captions.txt


In [10]:
files.download("video_frame_captions.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!pip install gTTS


In [12]:
from gtts import gTTS
from IPython.display import Audio

caption_text = "A dog playing with a ball in the garden."  # Replace with your caption
tts = gTTS(text=caption_text, lang='en')
tts.save("caption_audio.mp3")

# Play audio in notebook
Audio("caption_audio.mp3")


In [13]:
for i, cap in captions:
    print(f"🖼️ Frame {i}: {cap}")

    tts = gTTS(text=cap, lang='en')
    filename = f"frame_{i}_audio.mp3"
    tts.save(filename)

    display(Audio(filename))


🖼️ Frame 0: a rabbit is sitting in a hole in the grass


🖼️ Frame 5: a rabbit is sitting in a hole in the grass


🖼️ Frame 10: a small animal in the middle of a grassy field


🖼️ Frame 15: a small animal in the middle of a grassy field


🖼️ Frame 20: an animal in the middle of a grassy field


...

🖼️ Frame 110: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated


🖼️ Frame 115: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated


🖼️ Frame 120: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated


🖼️ Frame 125: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated


🖼️ Frame 130: a scene from the animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated animated


In [14]:
Audio("caption_audio.mp3", autoplay=True)


In [15]:
with open("all_captions.txt", "w") as f:
    for i, cap in captions:
        f.write(f"Frame {i}: {cap}\n")

from google.colab import files
files.download("all_captions.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>